In [56]:
# попробуем получить дополнительные фичи, исходя из чужих файликов

In [57]:
import pandas as pd
import numpy as np


In [58]:
df = pd.read_parquet("test_dataset_hackaton2023_test.gzip")

In [59]:
def filter_by_keyword(df, keywords_list):
    df = df[df['dish_name'].str.lower().str.contains('|'.join([keyword.lower() for keyword in keywords_list]))]
    return df

In [60]:
keywordsList = ["Латте", "Flash", "Кола", "Липтон", "Эвервесс", "Адреналин", "Эвервесс", "Капучино", "Кофе", "Дюшес", "чай", "Байкал", "Вода", "Шейк", "Фрустайл", "Какао", "Пиво", "Пепси", "Лимонад", "Мандарин", "Сок", "Балтика", "Эспрессо", "Миринда", "Севен", "Энергет.нап."]

values = [0.2, 0.3, 0.4, 0.5, 0.8,1]

condition = "Латте|Flash|Кола|Липтон|Эвервесс|Адреналин|Эвервесс|Капучино|Кофе|Дюшес|чай|Байкал|Вода|Шейк|Фрустайл|Какао|Пиво|Пепси|Лимонад|Мандарин|Сок|Балтика|Эспрессо|Миринда|Севен|Энергет.нап"

df['drink_size'] = np.select(
    [ df['dish_name'].str.contains(condition) & df['dish_name'].str.lower().str.contains("0,2"),
      df['dish_name'].str.contains(condition) & df['dish_name'].str.lower().str.contains("0,3"),
      df['dish_name'].str.contains(condition) & df['dish_name'].str.lower().str.contains("0,4|мал"),
      df['dish_name'].str.contains(condition) & df['dish_name'].str.lower().str.contains("0,5|станд"),
      df['dish_name'].str.contains(condition) & df['dish_name'].str.lower().str.contains("0,8|бол"),
      df['dish_name'].str.contains(condition) & df['dish_name'].str.lower().str.contains("1")], values, default=0)

In [61]:
df_by_customer = df['customer_id'].drop_duplicates()

In [62]:
df_drinksize_by_customer = df.groupby(['customer_id']).agg({'drink_size': ['std', 'mean']})

In [63]:
df_drinksize_by_customer.columns = get_column_names_for_grouping_df({'drink_size': ['std', 'mean']})

## Тип ресторана

In [64]:
df['with_toilet'] = np.select(
    [ df['format_name'].str.lower().str.contains("без туалета"),
      df['format_name'].str.lower().str.contains("с туалетом")], [0, 1], default=1)

df['food_court'] = np.select(
    [ df['format_name'].str.contains("Отдельно"),
      df['format_name'].str.contains("Фудкорт")], [0, 1], default=0)

df['countable'] = np.select(
    [ df['dish_name'].str.lower().str.contains("шт")], [1], default=0)

df['external_area'] = np.select(
    [ df['format_name'].str.contains("с внешней зоной"),
      df['format_name'].str.contains("без внешней зоны")], [0, 1], default=0)

In [65]:
df_restaurant_type_by_customer = df.groupby(['customer_id']).agg({'with_toilet': ['std', 'mean'],
                                                                  'food_court': ['std', 'mean'],
                                                                  'countable': ['std', 'mean'],
                                                                  'external_area': ['std', 'mean']})

In [66]:
def get_column_names_for_grouping_df(agg):
    column_names = []

    for key, value in agg.items():
        for v in value:
            column_names.append(f'{key}_{v}')

    return column_names

In [67]:
df_restaurant_type_by_customer.columns = get_column_names_for_grouping_df({'with_toilet': ['std', 'mean'],
                                                                  'food_court': ['std', 'mean'],
                                                                  'countable': ['std', 'mean'],
                                                                  'external_area': ['std', 'mean']})

# Тип еды

In [68]:
df['dish_name_filtered'] = df['dish_name']\
    .replace('\d+', '', regex=True)\
    .replace('\(|\)', '', regex=True)\
    .replace('в бутылке|без газа|станд|мал|бол|,|т3', '', regex=True)\
    .str.lower()\
    .replace('up ', '')\
    .replace('.', '')\
    .replace('1rub', '')

In [69]:
df['dish_name_filtered']

1773606             липтон лимон  л
4035564     пиво балтика  .л розлив
2507770                   кинг фри 
3963775    цезарь ролл со стрипсами
859943            up гранд чиз фреш
                     ...           
1677742       сырные медальоны  шт.
4450230             липтон грин дж 
3882998                 воппер ролл
1356999      двойной воппер с сыром
3964741                   гранд чиз
Name: dish_name_filtered, Length: 2498034, dtype: object

In [70]:
drinks = condition.lower()
drinks

'латте|flash|кола|липтон|эвервесс|адреналин|эвервесс|капучино|кофе|дюшес|чай|байкал|вода|шейк|фрустайл|какао|пиво|пепси|лимонад|мандарин|сок|балтика|эспрессо|миринда|севен|энергет.нап'

In [71]:
burger = 'чиз|бургер|чикен|цезарь|кинг'

In [72]:
potato = 'карт|фри|медальон'

In [73]:
sauce = 'соус'

In [74]:
whopper = 'ролл|воппер|твист'

In [75]:
df['is_burger'] = np.select([df['dish_name_filtered'].str.contains(burger)], [1], default=0)
df['is_drink'] = np.select([df['dish_name_filtered'].str.contains(drinks)], [1], default=0)
df['is_potato'] = np.select([df['dish_name_filtered'].str.contains(potato)], [1], default=0)
df['is_whopper'] = np.select([df['dish_name_filtered'].str.contains(whopper)], [1], default=0)
df['is_sauce'] = np.select([df['dish_name_filtered'].str.contains(sauce)], [1], default=0)

In [76]:
df_dishtype_by_customer = df.groupby(['customer_id']).agg({
    'is_drink': ['mean', 'std'],
    'is_burger': ['mean', 'std'],
    'is_potato': ['mean', 'std'],
    'is_whopper': ['mean', 'std'],
    'is_sauce': ['mean', 'std'],
})

In [77]:
column_names = []

for key, value in {
    'is_drink': ['mean', 'std'],
    'is_burger': ['mean', 'std'],
    'is_potato': ['mean', 'std'],
    'is_whopper': ['mean', 'std'],
    'is_sauce': ['mean', 'std'],
}.items():
    for v in value:
        column_names.append(f'{key}_{v}')

In [78]:
column_names

['is_drink_mean',
 'is_drink_std',
 'is_burger_mean',
 'is_burger_std',
 'is_potato_mean',
 'is_potato_std',
 'is_whopper_mean',
 'is_whopper_std',
 'is_sauce_mean',
 'is_sauce_std']

In [79]:
df_dishtype_by_customer.columns = column_names

In [80]:
from functools import reduce

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['customer_id'],
                                            how='outer'), [df_drinksize_by_customer, df_dishtype_by_customer, df_restaurant_type_by_customer])

In [87]:
df_merged.reset_index().to_parquet('test_additional_feats.parquet')

In [82]:
df_merged

,drink_size_std,drink_size_mean,is_drink_mean,is_drink_std,is_burger_mean,is_burger_std,is_potato_mean,is_potato_std,is_whopper_mean,is_whopper_std,is_sauce_mean,is_sauce_std,with_toilet_std,with_toilet_mean,food_court_std,food_court_mean,countable_std,countable_mean,external_area_std,external_area_mean
customer_id,,,,,,,,,,,,,,,,,,,,
52341,0.209149,0.117857,0.250000,0.440959,0.285714,0.460044,0.142857,0.356348,0.000000,0.000000,0.142857,0.356348,0.000000,1.00000,0.000000,0.00000,0.356348,0.142857,0.000000,0.00000
69175,0.156492,0.073469,0.183673,0.391230,0.306122,0.465657,0.183673,0.391230,0.081633,0.276642,0.183673,0.391230,0.000000,1.00000,0.000000,0.00000,0.199915,0.040816,0.421570,0.22449
73427,0.098349,0.028125,0.187500,0.393398,0.250000,0.436436,0.140625,0.350382,0.187500,0.393398,0.218750,0.416667,0.175368,0.96875,0.365963,0.15625,0.350382,0.140625,0.175368,0.03125
134577,0.155700,0.066667,0.166667,0.389249,0.000000,0.000000,0.166667,0.389249,0.333333,0.492366,0.250000,0.452267,0.000000,0.00000,0.000000,1.00000,0.452267,0.250000,0.000000,0.00000
156357,0.262630,0.130769,0.230769,0.438529,0.461538,0.518875,0.153846,0.375534,0.000000,0.000000,0.153846,0.375534,0.000000,1.00000,0.000000,1.00000,0.375534,0.153846,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46540272,0.245176,0.230000,0.500000,0.527046,0.100000,0.316228,0.100000,0.316228,0.000000,0.000000,0.100000,0.316228,0.000000,0.00000,0.000000,1.00000,0.483046,0.300000,0.000000,0.00000
46569275,0.167649,0.091667,0.333333,0.492366,0.333333,0.492366,0.166667,0.389249,0.166667,0.389249,0.000000,0.000000,0.452267,0.75000,0.452267,0.25000,0.389249,0.166667,0.000000,0.00000
46601088,0.176777,0.062500,0.125000,0.353553,0.500000,0.534522,0.375000,0.517549,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.00000,0.353553,0.125000,0.000000,0.00000


In [83]:
dd = pd.read_parquet('train_additional_feats.parquet')

In [88]:
dd.reset_index().to_parquet('train_additional_feats.parquet')

In [89]:
ddd = pd.read_parquet('test_additional_feats.parquet')

In [90]:
ddd

,customer_id,drink_size_std,drink_size_mean,is_drink_mean,is_drink_std,is_burger_mean,is_burger_std,is_potato_mean,is_potato_std,is_whopper_mean,...,is_sauce_mean,is_sauce_std,with_toilet_std,with_toilet_mean,food_court_std,food_court_mean,countable_std,countable_mean,external_area_std,external_area_mean
0,52341,0.209149,0.117857,0.250000,0.440959,0.285714,0.460044,0.142857,0.356348,0.000000,...,0.142857,0.356348,0.000000,1.00000,0.000000,0.00000,0.356348,0.142857,0.000000,0.00000
1,69175,0.156492,0.073469,0.183673,0.391230,0.306122,0.465657,0.183673,0.391230,0.081633,...,0.183673,0.391230,0.000000,1.00000,0.000000,0.00000,0.199915,0.040816,0.421570,0.22449
2,73427,0.098349,0.028125,0.187500,0.393398,0.250000,0.436436,0.140625,0.350382,0.187500,...,0.218750,0.416667,0.175368,0.96875,0.365963,0.15625,0.350382,0.140625,0.175368,0.03125
3,134577,0.155700,0.066667,0.166667,0.389249,0.000000,0.000000,0.166667,0.389249,0.333333,...,0.250000,0.452267,0.000000,0.00000,0.000000,1.00000,0.452267,0.250000,0.000000,0.00000
4,156357,0.262630,0.130769,0.230769,0.438529,0.461538,0.518875,0.153846,0.375534,0.000000,...,0.153846,0.375534,0.000000,1.00000,0.000000,1.00000,0.375534,0.153846,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112329,46540272,0.245176,0.230000,0.500000,0.527046,0.100000,0.316228,0.100000,0.316228,0.000000,...,0.100000,0.316228,0.000000,0.00000,0.000000,1.00000,0.483046,0.300000,0.000000,0.00000
112330,46569275,0.167649,0.091667,0.333333,0.492366,0.333333,0.492366,0.166667,0.389249,0.166667,...,0.000000,0.000000,0.452267,0.75000,0.452267,0.25000,0.389249,0.166667,0.000000,0.00000
112331,46601088,0.176777,0.062500,0.125000,0.353553,0.500000,0.534522,0.375000,0.517549,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,1.00000,0.353553,0.125000,0.000000,0.00000
112332,46632765,0.326599,0.133333,0.166667,0.408248,0.500000,0.547723,0.166667,0.408248,0.333333,...,0.000000,0.000000,0.000000,1.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000


In [91]:
www = pd.read_parquet('test_with_feats_v2.parquet')

In [92]:
www

,customer_id,receipt_sum_mean,receipt_sum_std,receipt_count,items_receipt_mean,items_receipt_mean_std,sum_delta_min,sum_delta_max,sum_delta_mean,sum_delta_std,...,second_half_max,second_half_min,second_half_std,diff,ratio,cosine_similarity,was_promo_mean,was_promo_std,was_1rub_mean,was_1rub_std
0,52341,563.942500,65.700330,4,7.000000,0.816497,-50.01,155.99,35.326667,107.447569,...,655.93,499.94,110.301587,27.985000,1.050886,0.109694,0.750000,0.500000,0.000000,0.000000
1,69175,430.306667,62.142941,9,5.444444,0.726483,-179.98,80.98,-10.498750,82.377989,...,480.96,374.98,47.429064,-11.098500,0.974572,0.083715,1.000000,0.000000,0.000000,0.000000
2,73427,254.240323,218.360745,31,2.064516,2.112431,-719.96,699.97,9.666667,349.223379,...,669.98,1.00,209.570345,32.792167,1.138180,0.036133,0.774194,0.425024,0.032258,0.179605
3,134577,372.970000,310.195853,4,3.000000,2.708013,-218.02,689.94,133.313333,487.551900,...,799.93,109.99,487.861253,163.980000,1.563544,0.180556,0.750000,0.500000,0.000000,0.000000
4,156357,558.963333,245.551523,3,4.333333,1.154701,-362.98,467.97,52.495000,587.570380,...,749.95,281.98,330.904760,-128.995000,0.799995,0.076923,0.666667,0.577350,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112329,46540272,441.633333,324.487289,3,3.333333,2.516611,-384.97,-259.98,-322.475000,88.381277,...,399.97,139.99,183.833621,-514.960000,0.343950,0.140000,0.333333,0.577350,0.000000,0.000000
112330,46569275,377.313333,390.697752,3,4.000000,4.358899,-430.98,-348.98,-389.980000,57.982756,...,349.98,1.00,246.766124,-605.470000,0.224711,0.097222,0.666667,0.577350,0.000000,0.000000
112331,46601088,193.316667,101.526598,3,2.666667,2.081666,-165.00,-19.99,-92.495000,102.537554,...,144.98,124.99,14.135065,-174.995000,0.435464,0.125000,0.333333,0.577350,0.000000,0.000000
112332,46632765,246.650000,40.409983,3,2.000000,1.000000,-29.99,79.99,25.000000,77.767604,...,289.98,209.99,56.561471,10.005000,1.041691,0.222222,0.333333,0.577350,0.000000,0.000000
